In [1]:
import pandas as pd
import jieba
import re
from gensim.models import word2vec
import gensim
import logging
path ='/Users/admin/Documents/Dataset/' 

In [2]:
# 载入数据，csv文件一般是逗号分隔的数据集
data = pd.read_csv(path +'online_shopping_10_cats.csv') 

In [3]:
# 提取“水果类”的评论用作本次实验
data_fruit = data[data.cat == '水果']

In [4]:
# 中文字符提取函数，仅提取评论中的中文字符，忽略标点符号以及英文字母、单词等
def extractChinese(s):
    pattern="[\u4e00-\u9fa5]+"#中文正则表达式
    regex = re.compile(pattern) #生成正则对象 
    results = regex.findall(s) #匹配
    return "". join(results)

In [5]:
# 将原始评论（仅含中文字符）进行分词，最终获得一个corpus列表，列表中的每一项是每个评论对应的分词列表
corpus = []
for rev in data_fruit.review:
    rev = extractChinese(rev)
    corpus.append(list(jieba.cut(rev)))
print(corpus[0])

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/f7/94hwsc710mscyzml4tk3x7_40000gn/T/jieba.cache
Loading model cost 0.613 seconds.
Prefix dict has been built succesfully.


['好吃', '下次', '就', '买宏辉', '果蔬', '的', '了', '的', '比', '的', '好些', '甜', '一分钱', '一分货', '日期', '新鲜']


In [6]:
# 训练并保存doc2vec模型
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

model_path = './fruit_review_d2v.model' #模型保存路径
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(corpus)] #为corpus数据集打标签
model = Doc2Vec(documents, vector_size=300, window=2, min_count=1, workers=4)
model.save(model_path)
model_d2v = Doc2Vec.load('./fruit_review_d2v.model')

In [7]:
# 使用训练好的模型将所有的review表示为向量,并从原始数据集中获取评论的标签
vectors = []
for review in corpus:
    vectors.append(model_d2v.infer_vector(review))

In [8]:
from sklearn.model_selection import train_test_split
x = vectors # 转化为向量后的数据集
y = list(data_fruit.label) #数据集的标签（好评1/差评0）
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=1) #将预处理后的数据集分割为训练集和测试集

In [9]:
#  建立和训练模型、使用模型并计算模型预测准确率
from sklearn.linear_model import LogisticRegression
import numpy as np
LRmodel = LogisticRegression() #建立logistic回归模型
LRmodel.fit(x_train,y_train) # 使用训练集训练模型
y_hat = LRmodel.predict(x_test) # 使用测试集进行类别预测
result = y_hat == y_test # 记录测试集中预测准确的数量
acc = np.mean(result) # 计算模型预测准确率
print(acc)

0.7488
